In [ ]:
# !pip install opentelemetry-api
# !pip install opentelemetry-sdk

In [ ]:
from pathlib import Path
import sys

# Add base dir to path to be able to access test folder.
base_dir = Path().cwd().parent.parent.resolve()
if str(base_dir) not in sys.path:
    print(f"Adding {base_dir} to sys.path")
    sys.path.append(str(base_dir))

In [ ]:
import logging

root = logging.getLogger()
root.setLevel(logging.DEBUG)
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
handler.addFilter(logging.Filter("trulens"))
formatter = logging.Formatter(
    "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
handler.setFormatter(formatter)
root.addHandler(handler)

In [ ]:
from trulens.experimental.otel_tracing.core.instrument import instrument


class TestApp:
    @instrument()
    def respond_to_query(self, query: str) -> str:
        return f"answer: {self.nested(query)}"

    @instrument(attributes={"nested_attr1": "value1"})
    def nested(self, query: str) -> str:
        return f"nested: {self.nested2(query)}"

    @instrument(
        attributes=lambda ret, exception, *args, **kwargs: {
            "nested2_ret": ret,
            "nested2_args[0]": args[0],
        }
    )
    def nested2(self, query: str) -> str:
        nested_result = ""

        try:
            nested_result = self.nested3(query)
        except Exception:
            pass

        return f"nested2: {nested_result}"

    @instrument(
        attributes=lambda ret, exception, *args, **kwargs: {
            "nested3_ex": exception.args if exception else None,
            "nested3_ret": ret,
            "selector_name": "special",
            "cows": "moo",
        }
    )
    def nested3(self, query: str) -> str:
        if query == "throw":
            raise ValueError("nested3 exception")
        return "nested3"

In [ ]:
import dotenv
from trulens.core.session import TruSession
from trulens.experimental.otel_tracing.core.init import init

dotenv.load_dotenv()

session = TruSession()
session.experimental_enable_feature("otel_tracing")
session.reset_database()
init(session, debug=True)

In [ ]:
from trulens.apps.custom import TruCustomApp

test_app = TestApp()
custom_app = TruCustomApp(test_app)

with custom_app as recording:
    test_app.respond_to_query("test")

with custom_app as recording:
    test_app.respond_to_query("throw")